In [82]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding
import base64
import more_itertools as mit
import os

Challenge 9

PKCS#7, pad message to equal to multiple of block size, the pad value will be the pad length.

To unpad, simply read the pad value and chop off the pad.


In [76]:
def pkcs_7_pad(b: bytearray, blocksize):
  padding_length = blocksize - (len(b) % blocksize) if blocksize > 0 else 0
  return b+bytearray([padding_length]*padding_length)

def pkcs_7_unpad(b: bytearray):
  return b[:-b[-1]]

In [77]:
v = pkcs_7_pad(b"YELLOW SUBMARINE", 20)
o = pkcs_7_unpad(v)
v, o

(b'YELLOW SUBMARINE\x04\x04\x04\x04', b'YELLOW SUBMARINE')

Challenge 10

In [78]:
def aes_encrypt_128_block(key:bytearray, plaintext:bytearray):
  backend = default_backend()
  cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=backend)
  encryptor = cipher.encryptor()
  return encryptor.update(plaintext) + encryptor.finalize()

def aes_decrypt_128_block(key:bytearray, ciphertext:bytearray):
  backend = default_backend()
  cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=backend)
  decryptor = cipher.decryptor()
  return decryptor.update(ciphertext) + decryptor.finalize()

In [79]:
with open("10.txt", "r") as f:
  cipher_text = base64.b64decode(f.read())
cipher_text

b'\t\x120\xaa\xde>\xb30\xdb\xaaCX\xf8\x8d*l\xd5\xcf\x83U\xcbh#9z\xd49\x06\xdfCDU\x7f\xc4\x83v\x93\xc1\xa8\xee;@\xac\xb22?\xad9oN\xf5\x0c\xbf\x02\xf8S\xd8Hs\x97>C\x0c0S\xc0*o\x8d\xb2\xed\'\x08\x13\x10V\xdff\x96[\x87mQ?\xca^\x95h\x10\xa36\xe3\x86\xbcv}Y\x8b\xed\xe7[\x91\xfeY%e\x9d\x0en\xa0\xf9Q\xa0\xb5\xae\xeaY\xc0!\n\xe2\x92\x16\x7f\xa2P\xe2\x94\xf2><\xa2>\xd2\x97\x83\x9e\x055\x0b\xdbT\x81\xde\xa2\xd5\x06\xdaA\xc4\xbcq\x92\xff\r\x93\xb7\x818\xdb\x110U\xf8\xb6\xf4\x9d\xf2\xa7C\xbfM\xc2\xfbj\x84Y!\xc2c1^*\xc5\x8f\xee\xbd\x92\xc6\xb2:5\xfc\x8c\xa1\xb5\nw\xbd\x8d\'K2c\x1a\x06\x8bazoF\x7f\x13h\x07\x04\xb4}\xdc\xa9\x97\xa7\x06\x8a1\x04}\xcc\xc4\xd8\xc3\xfd5_\xdc\xd6\xac\xb4E\x86\x9e\xa7\x86\xa7\xed\x03\xf5\xb41\xffr\xf4\xfb\x14\xdf\xfaM\x95\x8c\x10\xebW\xd1\xcf\\\xd1\x0bF\x03\xcd\xd0\xc9M\xa3\xbfi\xda\x010Z7\xd3o\x1d\x8al^\xa1\xb6\x0b\x1b\x1a\x00\x8cg\xaf\xe2\n\xc8r\xdd\x1f?i\xbcd\x97\xc7\x8a\xfdW\xc2\x19\xf0:m\x04$\xaa\xcfun\x0b\x13\xd1\x181]%\xb3\x8e;\xe1\xe1\xe5\xd3J_1b\x83\x03\x08f\xb6\xa

In [138]:
def bxor(x:bytearray, y:bytearray):
  return bytearray([i^j for i,j in zip(x,y)])

def aes_encrypt_cbc(plaintext:bytearray, key:bytearray, iv:bytearray):
  prev_cipher_block = iv
  ciphertext = bytearray()
  for block in mit.chunked(pkcs_7_pad(plaintext, 16), 16):
    prev_cipher_block = aes_encrypt_128_block(key, bxor(block, prev_cipher_block))
    ciphertext += prev_cipher_block
  return ciphertext

def aes_decrypt_cbc(ciphertext:bytearray, key:bytearray, iv:bytearray):
  prev_cipher_block = iv
  plaintext = bytearray()
  for block in mit.chunked(ciphertext, 16):
    plaintext += bxor(prev_cipher_block, aes_decrypt_128_block(key, bytearray(block)))
    prev_cipher_block = block
  return pkcs_7_unpad(plaintext)

In [139]:
# aes_encrypt_cbc(key=b"YELLOW SUBMARINE", iv=b'\x00\x00\x00 &c')

In [142]:
plain = bytearray(os.urandom(150))
key = os.urandom(16)
iv = os.urandom(16)
c = aes_encrypt_cbc(plain, key, iv)
o = aes_decrypt_cbc(c, key, iv)
o == plain

True